### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [ ]:
SCRIPT_NAME = "Y2017M09D13_RH_EE_task_control_V01"
OUTPUT_VERSION = 1
OUTPUT_FILE_NAME = "detailed_tasks"


ec2_output_path = ("/volumes/data/{}/output_V{:02.0f}/").format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Output s3: " + s3_output_path +
      "\nOutput ec2: " + ec2_output_path)


In [ ]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

In [12]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

Y2019M05D03 UTC 16:41


# Settings

In [1]:
MAXTASKS = 390
CANCELTASKS = 1 # Cancels all pending tasks

In [2]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [5]:
ee.Initialize()

# Functions

In [6]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print(task)
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print('canceling %s' % task)
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print(i)
    return df
    


In [7]:
taskList = get_tasks()


In [8]:
type(taskList)

list

In [9]:
len(taskList)

2263

In [10]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
detailedTasks.to_csv(ec2_output_path + OUTPUT_FILE_NAME + ".csv")
detailedTasks.to_pickle(ec2_output_path + OUTPUT_FILE_NAME + ".pkl")

In [ ]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive

# DANGER ZONE

In [ ]:
detailedTasks

In [13]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)

<Task EXPORT_FEATURES: modis_Y2016_t28_timestamp1475798400000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2016_t28_timestamp1475798400000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2016_t27_timestamp1475107200000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2016_t27_timestamp1475107200000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2016_t26_timestamp1474416000000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2016_t26_timestamp1474416000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2016_t25_timestamp1473724800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2016_t25_timestamp1473724800000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2016_t24_timestamp1473033600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2016_t24_timestamp1473033600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2016_t23_timestamp1472342400000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2016_t23_timestamp1472342400000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2016_t22_t

canceling <Task EXPORT_FEATURES: modis_Y2015_t07_timestamp1429056000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2015_t06_timestamp1428364800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2015_t06_timestamp1428364800000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2015_t05_timestamp1427673600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2015_t05_timestamp1427673600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2015_t04_timestamp1426982400000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2015_t04_timestamp1426982400000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2015_t03_timestamp1426291200000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2015_t03_timestamp1426291200000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2015_t02_timestamp1425600000000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2015_t02_timestamp1425600000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2015_t01_timestamp1424908800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y

<Task EXPORT_FEATURES: modis_Y2013_t15_timestamp1371513600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2013_t15_timestamp1371513600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2013_t14_timestamp1370822400000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2013_t14_timestamp1370822400000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2013_t13_timestamp1370131200000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2013_t13_timestamp1370131200000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2013_t12_timestamp1369440000000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2013_t12_timestamp1369440000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2013_t11_timestamp1368748800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2013_t11_timestamp1368748800000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2013_t10_timestamp1368057600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2013_t10_timestamp1368057600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2013_t09_t

canceling <Task EXPORT_FEATURES: modis_Y2011_t24_timestamp1314576000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2011_t23_timestamp1313884800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2011_t23_timestamp1313884800000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2011_t22_timestamp1313193600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2011_t22_timestamp1313193600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2011_t21_timestamp1312502400000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2011_t21_timestamp1312502400000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2011_t20_timestamp1311811200000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2011_t20_timestamp1311811200000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2011_t19_timestamp1311120000000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2011_t19_timestamp1311120000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2011_t18_timestamp1310428800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y

<Task EXPORT_FEATURES: modis_Y2010_t02_timestamp1267833600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2010_t02_timestamp1267833600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2010_t01_timestamp1267142400000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2010_t01_timestamp1267142400000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2010_t00_timestamp1266451200000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2010_t00_timestamp1266451200000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2009_t29_timestamp1254960000000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2009_t29_timestamp1254960000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2009_t28_timestamp1254268800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2009_t28_timestamp1254268800000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2009_t27_timestamp1253577600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2009_t27_timestamp1253577600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2009_t26_t

canceling <Task EXPORT_FEATURES: modis_Y2008_t11_timestamp1210896000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2008_t10_timestamp1210204800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2008_t10_timestamp1210204800000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2008_t09_timestamp1209513600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2008_t09_timestamp1209513600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2008_t08_timestamp1208822400000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2008_t08_timestamp1208822400000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2008_t07_timestamp1208131200000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2008_t07_timestamp1208131200000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2008_t06_timestamp1207440000000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2008_t06_timestamp1207440000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2008_t05_timestamp1206748800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y

<Task EXPORT_FEATURES: modis_Y2006_t19_timestamp1153353600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2006_t19_timestamp1153353600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2006_t18_timestamp1152662400000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2006_t18_timestamp1152662400000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2006_t17_timestamp1151971200000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2006_t17_timestamp1151971200000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2006_t16_timestamp1151280000000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2006_t16_timestamp1151280000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2006_t15_timestamp1150588800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2006_t15_timestamp1150588800000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2006_t14_timestamp1149897600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2006_t14_timestamp1149897600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2006_t13_t

canceling <Task EXPORT_FEATURES: modis_Y2004_t28_timestamp1096416000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2004_t27_timestamp1095724800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2004_t27_timestamp1095724800000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2004_t26_timestamp1095033600000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2004_t26_timestamp1095033600000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2004_t25_timestamp1094342400000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2004_t25_timestamp1094342400000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2004_t24_timestamp1093651200000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2004_t24_timestamp1093651200000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2004_t23_timestamp1092960000000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y2004_t23_timestamp1092960000000_V08 (READY)>
<Task EXPORT_FEATURES: modis_Y2004_t22_timestamp1092268800000_V08 (READY)>
canceling <Task EXPORT_FEATURES: modis_Y